# Decision Making LCA Tool

Welcome to the program! This program will help you simulate LCA scenarios for both Traditional Manufacturing and Additive Manufacturing and enable you to compare them

In [230]:
#### Importing necessary libraries

# Libraries for calculations
import numpy as np

from datetime import datetime as date
import functools

# Libraries for displaying information
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interactive,interact, HBox, Layout,VBox
from IPython.display import display

## Defining Useful functions

### LCA Functions

#### Material Production Phase

In [231]:
def MaterialProduction(EmbodiedMat1, EmbodiedMat2, massTM, massAM):
    '''
    Description:
    ------------

    This function will calculate material production energy requirements and 
    CO2 emissions for both TM and AM simultaneously by multiplying the array 
    of embodied energy and carbon by the mass relevant for the given path
    
    Variables:
    ----------
    
    EmbodiedMat1: 
    - Type: Array
    - Desc: Array with the embodied energy (MJ/kg) and carbon (kgCO2/kg) of the 
    material used in TM,  - [Energy, Carbon].
    
    EmbodiedMat2: 
    - Type: Array
    - Desc: Array with the embodied energy (MJ/kg) and carbon (kgCO2/kg) of the 
    material used in AM,  - [Energy, Carbon].
    
    massTM: 
    - Type: Float
    - Desc: Stores the mass of the initial workpiece that will be used in TM path.
    
    massAM: 
    - Type: Float
    - Desc: Stores the mass of the mass that will be placed on the AM path (has 
    to consider support mass and allowance of the machine)
    
    Returns:
    --------
    
    EG_TM:
    - Type: Array
    - Desc: Stores the energy and carbon demand for the material production for 
    the workpiece in TM
    
    EG_AM:
    - Type: Array
    - Desc: Stores the energy and carbon demand for the material production of 
    the material that will be placed in AM
    
    
    '''
    # Simply mutiplying the array of embodied impacts by the initial mass of both production paths. 
    EG_TM = np.multiply(EmbodiedMat1,massTM)
    EG_AM = np.multiply(EmbodiedMat2,massAM)
    return EG_TM, EG_AM

#### Manufacturing Phase

In [233]:
def Manufacturing(processing_steps):
    '''
    Description:
    ------------

    This function will calculate the energy and carbon cost of manufacturing in 
    each of the manufacturing paths by multiplying the Energy demand (MJ/kg) and 
    carbon emission (kgCO2/kg) for each step by the mass that will be removed/placed in each step
    
    Variables:
    ----------
    
    processing_steps: 
    - Type: Array
                                    
    - Desc: Each of the rows in the array ius meant to represent one step in the manufacturing 
    process, for example, if you have a milling step then a finishing step, the first row will 
    relate to the costs and mass removed of the milling step and the finishing step will be described 
    on the second row - it`s done in this format - 
                                    [[Specific Energy Demand, Specific CO2 Emission, Mass],
                                     [Specific Energy Demand, Specific CO2 Emission, Mass],
                                     [Specific Energy Demand, Specific CO2 Emission, Mass],
                                     [Specific Energy Demand, Specific CO2 Emission, Mass],
                                     [Specific Energy Demand, Specific CO2 Emission, Mass]]
    
        
    Returns:
    --------
    
    result:
    - Type: Array in this format ["Total Energy demand", "Total kgCO2 emitted"]
    - Desc: Stores the result of the multiplication and sum of specific impacts for the manufacturing steps
    
    
    '''

    # Multiplies the specific impacts (that are placed on the first two columns) by the mass relevant for 
    # that step (in the third column)
    result = np.multiply(processing_steps[:,:2],processing_steps[:,[2]])

    # Sums the total for each column, achieving the total energy demand and CO2 emission for the whole manufacturing phase
    result = np.sum(result, axis=0)

    return result


#### Transport Phase

In [234]:
# def Transport(distanceAM, distanceTM, methodAM, methodTM):
def Transport(mass, distance, method):
    '''
    Description:
    ------------
    Calculates the impacts of transportation of the manufactured produt until 
    the use phase. Simbolizes the distance travelled until an assembly step at 
    another place
    
    
    Variables:
    ----------
    mass: 
    - Type: Float
                                    
    - Desc: Simply stores the mass of the product that will be transported
    
    distance: 
    - Type: Float
                                    
    - Desc: Stores the distance that will be travelled between manufacturing and 
    use phase
    
    method: 
    - Type: String
                                    
    - Desc: Stores the methos of transportation, which then will be used as a key for a 
    dictionary in the fucntion to gather the transportation impacts per metric ton * km
    
    Returns:
    --------
    result: 
    - Type: String
                                    
    - Desc: Stores the methos of transportation, which then will be used as a key for a 
    dictionary in the fucntion to gather the transportation impacts per metric ton * km
    
    
    
    
    '''
    # DICTIONARY - MJ PER (METRIC TON*KM) 
    energyDemand = {'Ocean Shipping - Diesel': 0.16,
                    'Coastal Shipping - Diesel': 0.27,
                    'Barge - Diesel': 0.36,
                    'Rail - Diesel': 0.25, 
                    'Articulated HGV (up to 55 metric tons) - Diesel': 0.71, 
                    '40 metric ton truck - Diesel': 0.82, 
                    '32 metric ton truck - Diesel': 0.94, 
                    '14 metric ton truck - Diesel': 1.5, 
                    'Light goods vehicle - Diesel': 2.5, 
                    'Family car - Diesel': 1.7, 
                    'Family car - Gasoline': 2.6, 
                    'Family car -LPG': 3.9, 
                    'Family car - Hybrid Gasoline-electric': 1.55, 
                    'Super sports car and SUV - Gasoline': 4.8, 
                    'Long Haul Aircraft - Kerosene': 6.5, 
                    'Short Haul Aircraft - Kerosene': 13, 
                    'Helicopter (Eurocopter AS350) - Kerosene': 55}
    
    # DICTIONARY - KG OF CO2 PER (METRIC TON*KM)
    CO2Demand = {'Ocean Shipping - Diesel': 0.015,
                'Coastal Shipping - Diesel': 0.019,
                'Barge - Diesel': 0.028,
                'Rail - Diesel': 0.019, 
                'Articulated HGV (up to 55 metric tons) - Diesel': 0.05, 
                '40 metric ton truck - Diesel': 0.06, 
                '32 metric ton truck - Diesel': 0.067, 
                '14 metric ton truck - Diesel': 0.11, 
                'Light goods vehicle - Diesel': 0.18, 
                'Family car - Diesel': 0.12, 
                'Family car - Gasoline': 0.165, 
                'Family car -LPG': 0.18, 
                'Family car - Hybrid Gasoline-electric': 0.10, 
                'Super sports car and SUV - Gasoline': 0.31, 
                'Long Haul Aircraft - Kerosene': 0.45, 
                'Short Haul Aircraft - Kerosene': 0.76, 
                'Helicopter (Eurocopter AS350) - Kerosene': 3.3}
    
    # Initializing array with transport impacts using method as a key
    transportDemands = [energyDemand[method],CO2Demand[method]]
    
    # Multiplying the impacts by the products of mass and distance 
    # (important to note that since impacts are in metric tons we have 
    # to divide mass by 1000 since they are in kg)
    result = np.multiply(transportDemands,(mass/1000)*distance)
    
    return result

#### Use Phase

In [235]:
def Use(useType, dif_mass):
    '''
    Description:
    ------------

    This function will calculate the "gain" (the decrease of environmental cost 
    due to bigger efficiecies in geometry in AM) in energy demand and carbon emissions
    when the produced piece is in use.
    
    Variables:
    ----------
    
    useType: 
    - Type: String
    - Desc: Will be used as a key to the dictionaries of energy savings and carbon demand 
    decrease per kg of decreased weight
    
    dif_mass: 
    - Type: float
    - Desc: Stores the difference in mass between the mass of the finalized product 
    manufactured through TM and AM
    
    
    Returns:
    --------
    
    result:
    - Type: Array 
    - Desc: Stores the multiplicaion of the energy and a carbon gains in an array in 
    this format- [Energy Savings, CO2 savings]
    
    
    '''
    
    # DICTIONARY - MJ PER KG OF REDUCED WEIGHT
    energySaving = {'Long-Distance Aircraft': 200*(10**3),'Short-Distance Aircraft': 150*(10**3),
                    'Articulated Truck': 260,'Passenger Car - Diesel': 210, 'Passenger Car - Gasoline': 230}
    
    # DICTIONARY - KG OF CO2 PER KG OF REDUCED WEIGHT
    CO2Saving = {'Long-Distance Aircraft': 13.6*(10**3),'Short-Distance Aircraft': 10.2*(10**3),
                    'Articulated Truck': 18.5,'Passenger Car - Diesel': 14.9, 'Passenger Car - Gasoline': 15}
    
    
    # Creating the array that will be multiplied by the difference in mass, by using the key on the dictionaries
    valuesUse = [energySaving[useType],CO2Saving[useType]]
    
    # Mutiplies the array by the difference in mass arriving at the output
    result = np.multiply(valuesUse, dif_mass)
    
    return result

#### Disposal Phase

In [236]:
def Disposal(recyclability, emb_virgin, emb_waste, emb_rec, mass):
    '''
    Description:
    ------------

    Using the "Substitution Method" the environmental impact of the disposal is calculated
    
    Variables:
    ----------
    
    recyclability: 
    - Type: float
    - Desc: % of recyclability (combined recovery, recycling rate and recycling mateiral yield)
    
    emb_virgin: 
    - Type: Float
    - Desc: Stores the embodied impact from virgin material input per unit of material
    
    emb_waste: 
    - Type: Float
    - Desc: Stores the embodied impact for disposal of material per unit of material
    
    emb_rec: 
    - Type: Float
    - Desc: Stores the embodied impact from recycled material input per unit of material
    
    mass: 
    - Type: Float
    - Desc: Stores the mass that is being disposed of
    
    Returns:
    --------
    
    result:
    - Type: Array
    - Desc: 
    

    
    
    '''
    
    
    # Applying the substitution method to calculate the embodied impact of disposal
    E = np.multiply(emb_rec,recyclability)+np.multiply((emb_virgin+emb_waste),1-recyclability)

    # Multiplying array by the mass that goes to disposal
    result = np.multiply(E,mass)

    return result

### Control Functions
#### Calculation 
LCA Function, simulates the LCA by calling each of the phase fucntions accordingly

In [237]:
def Calculation(imassTM, 
                fmassTM,
                embTMe,
                embTMc, 
                embAMe,
                embAMc, 
                embERecTM,
                embCRecTM,
                embERecAM,
                embCRecAM,
                embEDisTM,
                embCDisTM,
                embEDisAM,
                embCDisAM,
                recAM, 
                recTM,
                TM11 ,
                TM12 ,
                TM13 ,
                TM21 ,
                TM22 ,
                TM23,
                TM31,
                TM32,
                TM33,
                TM41,
                TM42,
                TM43,
                TM51,
                TM52,
                TM53,
                AM11,
                AM12,
                AM13,
                AM21,
                AM22,
                AM23,
                AM31,
                AM32,
                AM33,
                AM41,
                AM42,
                AM43,
                AM51,
                AM52,
                AM53, 
                TMsteps, 
                AMsteps,
                secondary,
                mass_reduction_perc, 
                finishing_perc, 
                support_perc,
                eff_energ_conv,
                CO2equivalent,
                transpTM ,
                transpAM ,
                distTM ,
                distAM ,
                useType,
                boundry): 
    '''
    Description:
    ------------

    This is the core function of the program as it pertains to the calculation, 
    it controls and calls all the other functions into action. It has been separated 
    from the main function since it is called more than once, and in order to be more 
    efficient in code it was separated
    
    Variables:
    ----------
    
    imassTM: 
    - Type: float
    - Desc: Stores the initial mass of the TM workpiece
    fmassTM: 
    - Type: float
    - Desc: Stores the final mass after the workpiece has been finished in TM
    embTMe: 
    - Type: float
    - Desc: Stores the embodied energy impact for the material per unit of material 
    used in TM
    embTMc: 
    - Type: float
    - Desc: Stores the embodied CO2 impact for the material per unit of material 
    used in TM
    embAMe: 
    - Type: float
    - Desc: Stores the embodied energy impact for the material per unit of material 
    used in AM
    embAMc: 
    - Type: float
    - Desc: Stores the embodied CO2 impact for the material per unit of material 
    used in AM
    embEDisTM: 
    - Type: Float
    - Desc: Stores the embodied energy for disposal of material per unit of material used in TM
    embCDisTM: 
    - Type: Float
    - Desc: Stores the embodied carbon for disposal of material per unit of material used in TM
    embEDisAM: 
    - Type: Float
    - Desc: Stores the embodied energy for disposal of material per unit of material used in AM
    embCDisAM: 
    - Type: Float
    - Desc: Stores the embodied carbon for disposal of material per unit of material used in AM
    embERecTM: 
    - Type: Float
    - Desc: Stores the embodied energy impact from recycled material input per unit of material used in TM
    embCRecTM: 
    - Type: Float
    - Desc: Stores the embodied carbon from recycled material input per unit of material used in TM
    embERecAM: 
    - Type: Float
    - Desc: Stores the embodied energy impact from recycled material input per unit of material used in AM
    embCRecAM: 
    - Type: Float
    - Desc: Stores the embodied carbon from recycled material input per unit of material used in AM
    TM11: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in first step 
    of the TM path
    TM12: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in first step 
    of TM path
    TM13: 
    - Type: float
    - Desc: Stores the mass that will be removed in first step of the TM path
    TM21: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in second step 
    of the TM path
    TM22: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in second step 
    of TM path
    TM23: 
    - Type: float
    - Desc: Stores the mass that will be removed in second step of the TM path
    TM31: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in third step 
    of the TM path
    TM32: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in third step 
    of TM path
    TM33: 
    - Type: float
    - Desc: Stores the mass that will be removed in third step of the TM path
    TM41: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in fourth step 
    of the TM path
    TM42: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in fourth step 
    of TM path
    TM43: 
    - Type: float
    - Desc: Stores the mass that will be removed in fourth step of the TM path
    TM51: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in fifth step 
    of the TM path
    TM52: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in fifth step 
    of TM path
    TM53: 
    - Type: float
    - Desc: Stores the mass that will be removed in fifth step of the TM path
    TMsteps: 
    - Type: float
    - Desc: Stores the ammount of steps that shpuld be considerd in this 
    simulation for the TM path, as in if the number 3 is selected 3 
    steps will be considered, if 2 is selected only the first two steps will 
    be considered. Can go up to 5
    AM11: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in first step 
    of the AM path
    AM12: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in first step 
    of AM path
    AM13: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in first step of the AM path
    AM21: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in second step 
    of the AM path
    AM22: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in second step 
    of AM path
    AM23: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in second step of the AM path
    AM31: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in third step 
    of the AM path
    AM32: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in third step 
    of AM path
    AM33: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in third step of the AM path
    AM41: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in fourth step 
    of the AM path
    AM42: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in fourth step 
    of AM path
    AM43: 
    - Type: float
    - Desc: Stores the mass that will be removed in fourth step of the AM path
    TM51: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in fifth step 
    of the AM path
    AM52: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in fifth step 
    of AM path
    AM53: 
    - Type: float
    - Desc: Stores the mass that will be removed in fifth step of the AM path
    AMsteps: 
    - Type: float
    - Desc: Stores the ammount of steps that shpuld be considerd in this 
    simulation for the AM path, as in if the number 3 is selected all 3 
    steps will be considered, if 2 is selected only the first two steps will 
    be considered
    mass_reduction_perc: 
    - Type: float
    - Desc: Percentage of the final mass of the product produced through TM, 
    that will be reduced due to geometry optimization in AM path
    finishing_perc: 
    - Type: float
    - Desc: Percentage of additional mass that is placed in the AM process due 
    to machine allowance that will later need to be removed
    support_perc: 
    - Type: float
    - Desc: Percentage of additional mass that is placed in the AM process due 
    to the need to create supports during the build of a particular piece
    eff_energ_conv: 
    - Type: float
    - Desc: Stores the efficiency of the Power Grid considered in the simulation
    CO2equivalent: 
    - Type: float
    - Desc: Stores the equivalent CO2 emissions due to the energy required to 
    produce the piece in the considered Power Grid in the simulation
    
    
    recTM: 
    - Type: float
    - Desc: % of recyclability (combined recovery, recycling rate and recycling mateiral yield)
    
    
    recAM: 
    - Type: float
    - Desc: % of recyclability (combined recovery, recycling rate and recycling mateiral yield)
    
    secondary:
    - Type: boolean
    - Desc: Serves as a flag of whether or no the input for manufacturing steps is in primary or secondary use
    
    useType: 
    - Type: string
    - Desc: Stores the input of where the user wants to consider thepiece to be used in;
    
    transpTM:
    - Type: string
    - Desc: Stores the name of the transportation mode that will be used for TM, serves as a key for 
    the dictionary of specific impacts
    
    transpAM:
    - Type: string
    - Desc: Stores the name of the transportation mode that will be used for AM, serves as a key for 
    the dictionary of specific impacts
    
    distTM:
    - Type: float
    - Desc:  Stores the distance that will be travelled between manufacturing and 
    use phase in TM path
    
    distAM:
    - Type: float
    - Desc: Stores the distance that will be travelled between manufacturing and 
    use phase in AM path
    
    boundry:
    - Type: String
    - Desc: Stores boundry that will be considered for this simulation, depending 
    on its value graphs will display either a Cradle to grave, or Cradle to Gate or Gate to Gate scenario
    
    Returns:
    --------
    
    EnergyBarsTM:
    - Type: dictionary
    - Desc: Stores the energy impact result of each of the life cycle phases of the TM path
    
    CO2BarsTM:
    - Type: dictionary
    - Desc: Stores the CO2 impact result of each of the life cycle phases of the TM path
    
    EnergyBarsAM:
    - Type: dictionary
    - Desc: Stores the energy impact result of each of the life cycle phases of the AM path
    
    CO2BarsAM:
    - Type: dictionary
    - Desc: Stores the CO2 impact result of each of the life cycle phases of the AM path
    
    
    
    
    
    '''
    
    ############## MATERIAL PRODUCTION

    massAM = fmassTM*(1-mass_reduction_perc)/(1-(support_perc+finishing_perc))
    
    TMmat_prod, AMmat_prod = MaterialProduction([embTMe,embTMc] , [embAMe,embAMc] , imassTM, massAM)
    
    ############# MANUFACTURING
    
    removed_massTM = imassTM-fmassTM
    if secondary == False:
        TM_processing = np.array([[TM11,TM12,TM13*imassTM],[TM21,TM22,TM23*imassTM],[TM31,TM32,TM33*imassTM],[TM41,TM42,TM43*imassTM],[TM51,TM52,TM53*imassTM]])
        uTM_processing = TM_processing
        uTM_processing[TMsteps:] = 0

        AM_processing = np.array([[AM11,AM12,AM13*massAM],[AM21,AM22,AM23*massAM],[AM31,AM32,AM33*massAM],[AM41,AM42,AM43*massAM],[AM51,AM52,AM53*massAM]]) 
        uAM_processing = AM_processing
        uAM_processing[AMsteps:] = 0
    
    else:
        EnergyProcessTM = np.divide([TM11,TM21,TM31,TM41,TM51],eff_energ_conv)
        CO2ProcessTM = np.multiply([TM11,TM21,TM31,TM41,TM51],CO2equivalent)
        EnergyProcessAM = np.divide([AM11,AM21,AM31,AM41,AM51],eff_energ_conv)
        CO2ProcessAM = np.multiply([AM11,AM21,AM31,AM41,AM51],CO2equivalent)
    
        TM_processing = np.array([[EnergyProcessTM[0],CO2ProcessTM[0],TM13*imassTM],
                                  [EnergyProcessTM[1],CO2ProcessTM[1],TM23*imassTM],
                                  [EnergyProcessTM[2],CO2ProcessTM[2],TM33*imassTM],
                                  [EnergyProcessTM[3],CO2ProcessTM[3],TM43*imassTM],
                                  [EnergyProcessTM[4],CO2ProcessTM[4],TM53*imassTM]])
        uTM_processing = TM_processing
        uTM_processing[TMsteps:] = 0

        AM_processing = np.array([[EnergyProcessAM[0],CO2ProcessAM[0],AM13*massAM],
                                  [EnergyProcessAM[1],CO2ProcessAM[1],AM23*massAM],
                                  [EnergyProcessAM[2],CO2ProcessAM[2],AM33*massAM],
                                  [EnergyProcessAM[3],CO2ProcessAM[3],AM43*massAM],
                                  [EnergyProcessAM[4],CO2ProcessAM[4],AM53*massAM]]) 
        uAM_processing = AM_processing
        uAM_processing[AMsteps:] = 0
    
    
    
    TMmanuf = Manufacturing(uTM_processing)
    AMmanuf = Manufacturing(uAM_processing)
    
    ############# TRANSPORT
    
    TMtransport = Transport(fmassTM, distTM, transpTM)
    AMtransport = Transport(fmassTM*(1-mass_reduction_perc), distAM, transpAM)
    
    ############# USE
    
    reducedWeight = fmassTM*(-mass_reduction_perc) 
    AMuse = Use(useType, reducedWeight)
    
    ############# DISPOSAL
    
    
    TMdisposal = Disposal(recTM,np.array([embTMe,embTMc]),np.array([embEDisTM,embCDisTM]),np.array([embERecTM,embCRecTM]),imassTM)
    AMdisposal = Disposal(recAM,np.array([embTMe,embAMc]),np.array([embEDisAM,embCDisAM]),np.array([embERecAM,embCRecAM]),massAM)
    
    
    
    
    
    if boundry == 'Cradle to Grave': 
    
        EnergyBarsTM  = {'Material Production':TMmat_prod[0], 'Manufacturing':TMmanuf[0], 'Transport':TMtransport[0], 'Use':0, 'Disposal':TMdisposal[0]}
        CO2BarsTM = {'Material Production':TMmat_prod[1], 'Manufacturing':TMmanuf[1], 'Transport':TMtransport[1], 'Use':0, 'Disposal':TMdisposal[1]}

        EnergyBarsTM["Total"] = sum(EnergyBarsTM.values())
        CO2BarsTM["Total"] = sum(CO2BarsTM.values())

        EnergyBarsAM  = {'Material Production':AMmat_prod[0], 'Manufacturing':AMmanuf[0], 'Transport':AMtransport[0], 'Use':AMuse[0], 'Disposal':AMdisposal[0]}
        CO2BarsAM = {'Material Production':AMmat_prod[1], 'Manufacturing':AMmanuf[1], 'Transport':AMtransport[1], 'Use':AMuse[1], 'Disposal':AMdisposal[1]}

        EnergyBarsAM["Total"] = sum(EnergyBarsAM.values())
        CO2BarsAM["Total"] = sum(CO2BarsAM.values())
    
    elif boundry == 'Cradle to Gate':
        EnergyBarsTM  = {'Material Production':TMmat_prod[0], 'Manufacturing':TMmanuf[0]}
        CO2BarsTM = {'Material Production':TMmat_prod[1], 'Manufacturing':TMmanuf[1]}

        EnergyBarsTM["Total"] = sum(EnergyBarsTM.values())
        CO2BarsTM["Total"] = sum(CO2BarsTM.values())

        EnergyBarsAM  = {'Material Production':AMmat_prod[0], 'Manufacturing':AMmanuf[0]}
        CO2BarsAM = {'Material Production':AMmat_prod[1], 'Manufacturing':AMmanuf[1]}

        EnergyBarsAM["Total"] = sum(EnergyBarsAM.values())
        CO2BarsAM["Total"] = sum(CO2BarsAM.values())
    
    else:
        EnergyBarsTM  = {'Manufacturing':TMmanuf[0]}
        CO2BarsTM = {'Manufacturing':TMmanuf[1]}
        
        EnergyBarsTM["Total"] = sum(EnergyBarsTM.values())
        CO2BarsTM["Total"] = sum(CO2BarsTM.values())

        EnergyBarsAM  = {'Manufacturing':AMmanuf[0]}
        CO2BarsAM = {'Manufacturing':AMmanuf[1]}
        
        EnergyBarsAM["Total"] = sum(EnergyBarsAM.values())
        CO2BarsAM["Total"] = sum(CO2BarsAM.values())

    return (EnergyBarsTM,CO2BarsTM,EnergyBarsAM,CO2BarsAM)

#### Completed - Main function of the program
Organizes graph displays, connected with ipywidgets, runs solver and main simulation

In [240]:
def Completed(imassTM = 5.13, 
              fmassTM=2.04,
              embTMe=668,
              embTMc=39.5, 
              embAMe=723,
              embAMc=40, 
              embERecTM=500,
              embCRecTM=500,
              embERecAM=500,
              embCRecAM=500,
              embEDisTM=500,
              embCDisTM=500,
              embEDisAM=500,
              embCDisAM=500,
              recAM=0.15, 
              recTM=0.15,
              TM11 = 2.28,
              TM12 = 0.17,
              TM13 = 0.85,
              TM21 = 18.5,
              TM22 = 1.53,
              TM23 = 0.15,
              TM31 = 0,
              TM32 = 0,
              TM33 = 0,
              TM41 = 0,
              TM42 = 0,
              TM43 = 0,
              TM51 = 0,
              TM52 = 0,
              TM53 = 0,
              AM11 = 176.5,
              AM12 = 9.6,
              AM13 = 1,
              AM21 = 18.5,
              AM22 = 1.39,
              AM23 = 0.1,
              AM31 = 0,
              AM32 = 0,
              AM33 = 0,
              AM41 = 0,
              AM42 = 0,
              AM43 = 0,
              AM51 = 0,
              AM52 = 0,
              AM53 = 0, 
              TMsteps=2, 
              AMsteps=2,
              secondary = True,
              mass_reduction_perc=0, 
              finishing_perc=0.1, 
              support_perc = 0.2,
              eff_energ_conv = 0.4,
              CO2equivalent = 20,
              transpTM = 'Ocean Shipping - Diesel',
              transpAM = 'Ocean Shipping - Diesel',
              distTM = 300,
              distAM = 5,
              useType = 'Long-Distance Aircraft', 
              boundry='Cradle to Grave',
              solver = True, 
              gType = "All"):
    '''
    Description:
    ------------

    This function calls the Calculation function and is called by the ipywidgets 
    library to create the control interface for the program
    
    Variables:
    ----------
    
    imassTM: 
    - Type: float
    - Desc: Stores the initial mass of the TM workpiece
    fmassTM: 
    - Type: float
    - Desc: Stores the final mass after the workpiece has been finished in TM
    embTMe: 
    - Type: float
    - Desc: Stores the embodied energy impact for the material per unit of material 
    used in TM
    embTMc: 
    - Type: float
    - Desc: Stores the embodied CO2 impact for the material per unit of material 
    used in TM
    embAMe: 
    - Type: float
    - Desc: Stores the embodied energy impact for the material per unit of material 
    used in AM
    embAMc: 
    - Type: float
    - Desc: Stores the embodied CO2 impact for the material per unit of material 
    used in AM
    embEDisTM: 
    - Type: Float
    - Desc: Stores the embodied energy for disposal of material per unit of material used in TM
    embCDisTM: 
    - Type: Float
    - Desc: Stores the embodied carbon for disposal of material per unit of material used in TM
    embEDisAM: 
    - Type: Float
    - Desc: Stores the embodied energy for disposal of material per unit of material used in AM
    embCDisAM: 
    - Type: Float
    - Desc: Stores the embodied carbon for disposal of material per unit of material used in AM
    embERecTM: 
    - Type: Float
    - Desc: Stores the embodied energy impact from recycled material input per unit of material used in TM
    embCRecTM: 
    - Type: Float
    - Desc: Stores the embodied carbon from recycled material input per unit of material used in TM
    embERecAM: 
    - Type: Float
    - Desc: Stores the embodied energy impact from recycled material input per unit of material used in AM
    embCRecAM: 
    - Type: Float
    - Desc: Stores the embodied carbon from recycled material input per unit of material used in AM
    TM11: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in first step 
    of the TM path
    TM12: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in first step 
    of TM path
    TM13: 
    - Type: float
    - Desc: Stores the mass that will be removed in first step of the TM path
    TM21: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in second step 
    of the TM path
    TM22: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in second step 
    of TM path
    TM23: 
    - Type: float
    - Desc: Stores the mass that will be removed in second step of the TM path
    TM31: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in third step 
    of the TM path
    TM32: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in third step 
    of TM path
    TM33: 
    - Type: float
    - Desc: Stores the mass that will be removed in third step of the TM path
    TM41: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in fourth step 
    of the TM path
    TM42: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in fourth step 
    of TM path
    TM43: 
    - Type: float
    - Desc: Stores the mass that will be removed in fourth step of the TM path
    TM51: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in fifth step 
    of the TM path
    TM52: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in fifth step 
    of TM path
    TM53: 
    - Type: float
    - Desc: Stores the mass that will be removed in fifth step of the TM path
    TMsteps: 
    - Type: float
    - Desc: Stores the ammount of steps that shpuld be considerd in this 
    simulation for the TM path, as in if the number 3 is selected 3 
    steps will be considered, if 2 is selected only the first two steps will 
    be considered. Can go up to 5
    AM11: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in first step 
    of the AM path
    AM12: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in first step 
    of AM path
    AM13: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in first step of the AM path
    AM21: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in second step 
    of the AM path
    AM22: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in second step 
    of AM path
    AM23: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in second step of the AM path
    AM31: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in third step 
    of the AM path
    AM32: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in third step 
    of AM path
    AM33: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in third step of the AM path
    AM41: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in fourth step 
    of the AM path
    AM42: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in fourth step 
    of AM path
    AM43: 
    - Type: float
    - Desc: Stores the mass that will be removed in fourth step of the AM path
    TM51: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in fifth step 
    of the AM path
    AM52: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in fifth step 
    of AM path
    AM53: 
    - Type: float
    - Desc: Stores the mass that will be removed in fifth step of the AM path
    AMsteps: 
    - Type: float
    - Desc: Stores the ammount of steps that shpuld be considerd in this 
    simulation for the AM path, as in if the number 3 is selected all 3 
    steps will be considered, if 2 is selected only the first two steps will 
    be considered
    mass_reduction_perc: 
    - Type: float
    - Desc: Percentage of the final mass of the product produced through TM, 
    that will be reduced due to geometry optimization in AM path
    finishing_perc: 
    - Type: float
    - Desc: Percentage of additional mass that is placed in the AM process due 
    to machine allowance that will later need to be removed
    support_perc: 
    - Type: float
    - Desc: Percentage of additional mass that is placed in the AM process due 
    to the need to create supports during the build of a particular piece
    eff_energ_conv: 
    - Type: float
    - Desc: Stores the efficiency of the Power Grid considered in the simulation
    CO2equivalent: 
    - Type: float
    - Desc: Stores the equivalent CO2 emissions due to the energy required to 
    produce the piece in the considered Power Grid in the simulation
    
    
    recTM: 
    - Type: float
    - Desc: % of recyclability (combined recovery, recycling rate and recycling mateiral yield)
    
    
    recAM: 
    - Type: float
    - Desc: % of recyclability (combined recovery, recycling rate and recycling mateiral yield)
    
    secondary:
    - Type: boolean
    - Desc: Serves as a flag of whether or no the input for manufacturing steps is in primary or secondary use
    
    useType: 
    - Type: string
    - Desc: Stores the input of where the user wants to consider thepiece to be used in;
    
    transpTM:
    - Type: string
    - Desc: Stores the name of the transportation mode that will be used for TM, serves as a key for 
    the dictionary of specific impacts
    
    transpAM:
    - Type: string
    - Desc: Stores the name of the transportation mode that will be used for AM, serves as a key for 
    the dictionary of specific impacts
    
    distTM:
    - Type: float
    - Desc:  Stores the distance that will be travelled between manufacturing and 
    use phase in TM path
    
    distAM:
    - Type: float
    - Desc: Stores the distance that will be travelled between manufacturing and 
    use phase in AM path
    
    boundry:
    - Type: String
    - Desc: Stores boundry that will be considered for this simulation, depending 
    on its value graphs will display either a Cradle to grave, or Cradle to Gate or Gate to Gate scenario
   
    solver: 
    - Type: string
    - Desc: Serves as a flag from the user to indicate whether the breakeven percentage 
    of mass reduction from TM to AM should be calculated;
    
    
    gType: 
    - Type: string
    - Desc: Serves as an input from the user to decide the type of graph they want to see

    
    
    Returns:
    --------    
    
    Doesn't return anything but print out the desided plot
    
    '''
    # Calls Calculation function to receive valiues for each phase
    EnergyBarsTM,CO2BarsTM,EnergyBarsAM,CO2BarsAM = Calculation(imassTM, 
                                                                fmassTM,
                                                                embTMe,
                                                                embTMc, 
                                                                embAMe,
                                                                embAMc, 
                                                                embERecTM,
                                                                embCRecTM,
                                                                embERecAM,
                                                                embCRecAM,
                                                                embEDisTM,
                                                                embCDisTM,
                                                                embEDisAM,
                                                                embCDisAM,
                                                                recAM, 
                                                                recTM,
                                                                TM11 ,
                                                                TM12 ,
                                                                TM13 ,
                                                                TM21 ,
                                                                TM22 ,
                                                                TM23,
                                                                TM31,
                                                                TM32,
                                                                TM33,
                                                                TM41,
                                                                TM42,
                                                                TM43,
                                                                TM51,
                                                                TM52,
                                                                TM53,
                                                                AM11,
                                                                AM12,
                                                                AM13,
                                                                AM21,
                                                                AM22,
                                                                AM23,
                                                                AM31,
                                                                AM32,
                                                                AM33,
                                                                AM41,
                                                                AM42,
                                                                AM43,
                                                                AM51,
                                                                AM52,
                                                                AM53, 
                                                                TMsteps, 
                                                                AMsteps,
                                                                secondary,
                                                                mass_reduction_perc, 
                                                                finishing_perc, 
                                                                support_perc,
                                                                eff_energ_conv,
                                                                CO2equivalent,
                                                                transpTM ,
                                                                transpAM ,
                                                                distTM ,
                                                                distAM ,
                                                                useType,
                                                                boundry)
    
    
    
    ## INITIALIZING SOLVER TO FIND EQUILIBRIUM OF ENERGY IN FUCNTION MASS REDUCTION 
    mass_reduc_solver=0.1
    step = 0.1
    itera = 0
    last_result = 0
    # Initializes a loop that basically runs a Newton Method to find the correct value
    while solver and (itera<20):
        EnergyBarsTMs,CO2BarsTMs,EnergyBarsAMs,CO2BarsAMs = Calculation(imassTM, 
                                                                        fmassTM,
                                                                        embTMe,
                                                                        embTMc, 
                                                                        embAMe,
                                                                        embAMc, 
                                                                        embERecTM,
                                                                        embCRecTM,
                                                                        embERecAM,
                                                                        embCRecAM,
                                                                        embEDisTM,
                                                                        embCDisTM,
                                                                        embEDisAM,
                                                                        embCDisAM,
                                                                        recAM, 
                                                                        recTM,
                                                                        TM11 ,
                                                                        TM12 ,
                                                                        TM13 ,
                                                                        TM21 ,
                                                                        TM22 ,
                                                                        TM23,
                                                                        TM31,
                                                                        TM32,
                                                                        TM33,
                                                                        TM41,
                                                                        TM42,
                                                                        TM43,
                                                                        TM51,
                                                                        TM52,
                                                                        TM53,
                                                                        AM11,
                                                                        AM12,
                                                                        AM13,
                                                                        AM21,
                                                                        AM22,
                                                                        AM23,
                                                                        AM31,
                                                                        AM32,
                                                                        AM33,
                                                                        AM41,
                                                                        AM42,
                                                                        AM43,
                                                                        AM51,
                                                                        AM52,
                                                                        AM53, 
                                                                        TMsteps, 
                                                                        AMsteps,
                                                                        secondary,
                                                                        mass_reduc_solver, 
                                                                        finishing_perc, 
                                                                        support_perc,
                                                                        eff_energ_conv,
                                                                        CO2equivalent,
                                                                        transpTM ,
                                                                        transpAM ,
                                                                        distTM ,
                                                                        distAM ,
                                                                        useType,
                                                                        boundry)
        
        itera += 1
        if (EnergyBarsTMs["Total"] - EnergyBarsAMs["Total"] > 0) :
            result = -1 
        else: 
            result = +1
            
        if itera == 1:
            last_result = result
            
        if last_result != result:
            step = step/2
            
        mass_reduc_solver = mass_reduc_solver + result*step
        last_result = result
    
    # prints result from the solver
    print("Equilibrium of reduction in mass: ",mass_reduc_solver)
    
    
    ### START display code
    # Degree of rotation in xlabels
    rotation_deg = 30
    
    # Selects graphs according to user input, in this scenario 
    # shows graphs for Energy and CO2 for both TM and AM
    if gType == "All":

        fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, sharey='row')

        namesE = list(EnergyBarsTM.keys())
        valuesE = list(EnergyBarsTM.values())

        ax1.bar(namesE,valuesE, color='red')
        ax1.set_title('Traditional Manufacturing')
        ax1.set_ylabel('Energy Consumption (MJ)')
        ax1.tick_params(axis='x',labelsize=15, labelrotation=rotation_deg)

        namesC = list(CO2BarsTM.keys())
        valuesC = list(CO2BarsTM.values())

        ax3.bar(namesC,valuesC, color='red')
        ax3.set_ylabel('CO2 production (kg)')
        ax3.tick_params(axis='x',labelsize=15, labelrotation=rotation_deg)
        
        namesE = list(EnergyBarsAM.keys())
        valuesE = list(EnergyBarsAM.values())

        ax2.bar(namesE,valuesE, color='blue')
        ax2.set_title('Additive Manufacturing')
        ax2.set_ylabel('Energy Consumption (MJ)')
        ax2.tick_params(axis='x',labelsize=15, labelrotation=rotation_deg)

        namesC = list(CO2BarsAM.keys())
        valuesC = list(CO2BarsAM.values())

        ax4.bar(namesC,valuesC, color='blue')
        ax4.set_ylabel('CO2 production (kg)')
        ax4.tick_params(axis='x',labelsize=15, labelrotation=rotation_deg)

    # Shows results as the delta between TM and Am for each phase
    else:
        namesE = list(EnergyBarsTM.keys())
        valuesE = np.subtract(list(EnergyBarsTM.values()), list(EnergyBarsAM.values()))
        
        namesC = list(CO2BarsTM.keys())
        valuesC = np.subtract(list(CO2BarsTM.values()), list(CO2BarsAM.values()))
        
        fig, (ax1,ax2) = plt.subplots(1,2)
        ax1.bar(namesE,valuesE, color='red')
        ax1.set_title('Difference in Energy Consumption')
        ax1.set_ylabel('Delta in Energy Consumption (MJ)')
        ax1.tick_params(axis='x',labelsize=15, labelrotation=rotation_deg)

        
        ax2.bar(namesC,valuesC, color='green')
        ax2.set_title('Difference in CO2 Consumption')
        ax2.set_ylabel('Delta in CO2 production (kg)')
        ax2.tick_params(axis='x',labelsize=15, labelrotation=rotation_deg)


    
    fig.set_size_inches(18.5, 10.5)
    plt.rcParams.update({'font.size': 15})
    
    fig.tight_layout()
    plt.show(fig)
    
    # Prints results achieved for each phase
    print("Energy:\n")
    print("TM\n",EnergyBarsTM)
    print("AM\n",EnergyBarsAM)
    print("Carbon:\n")
    print("TM\n",CO2BarsTM)
    print("AM\n",CO2BarsAM)
        

#### Read Input
Very useful function ofr usability, not necessary for the program exactly but very welcome in practicality

In [241]:
def ReadInput(file_name):
    '''
    Description:
    ------------

    This function is designed to read a .txt file and extract the inputs to use in the simulation, 
    it is made to simply inputting numbers if you want to reproduce a previous simulation
    
    Variables:
    ----------
    
    file_name: 
    - Type: string
    - Desc: Stores the name of the file from which the inputs will be extracted

    
    
    Returns:
    --------    
    
    inputDict: 
    - Type: Dictionary
    - Desc: Dictionary of inputs, each key is the name of the variable and 
    the values are the input values
    
    '''
    # Initializes dictionary
    inputDict = {}
    # Opens desired file
    f = open(file_name,'r')
    
    # Loop to go thorugh each line one by one
    for line in f.readlines():
        
        # Separating variable name from value to add entry to dictionary
        words = line.split('=')
        key = words[0]
        value = words[1].replace('\n','').replace('"','').replace("'",'')
        
        # Tries to convert string into float, in case there is an error, 
        # it means that the variable type was supposed to be string as it alredy is and doesn't need any conversion
        try:
            value = float(value)
        except:
            pass
        
        inputDict[key]=value
    return inputDict

# Calls function to store initial input
I = ReadInput("input09-02-2023--18-13-40")
print(I)

{'imassTM': 1.515, 'fmassTM': 1.378, 'embTMe': 32.0, 'embTMc': 2.4, 'embAMe': 190.0, 'embAMc': 12.0, 'embERecTM': 8.5, 'embCRecTM': 0.6, 'embERecAM': 32.0, 'embCRecAM': 2.55, 'embEDisTM': 0.0, 'embCDisTM': 0.0, 'embEDisAM': 0.0, 'embCDisAM': 0.0, 'recAM': 0.9, 'recTM': 0.85, 'TM11': 10.0, 'TM12': 0.7, 'TM13': 1.0, 'TM21': 5.0, 'TM22': 0.4, 'TM23': 0.1, 'TM31': 0.0, 'TM32': 0.0, 'TM33': 0.0, 'TM41': 0.0, 'TM42': 0.0, 'TM43': 0.0, 'TM51': 0.0, 'TM52': 0.0, 'TM53': 0.0, 'AM11': 328.0, 'AM12': 9.6, 'AM13': 1.0, 'AM21': 18.5, 'AM22': 1.39, 'AM23': 0.1, 'AM31': 0.0, 'AM32': 0.0, 'AM33': 0.0, 'AM41': 0.0, 'AM42': 0.0, 'AM43': 0.0, 'AM51': 0.0, 'AM52': 0.0, 'AM53': 0.0, 'TMsteps': 2.0, 'AMsteps': 1.0, 'mass_reduction_perc': 0.7, 'finishing_perc': 0.1, 'support_perc': 0.2, 'eff_energ_conv': 0.4, 'CO2equivalent': 0.0711, 'transpTM': 'Ocean Shipping - Diesel', 'transpAM': 'Ocean Shipping - Diesel', 'distTM': 0.0, 'distAM': 0.0, 'useType': 'Passenger Car - Diesel', 'boundry': 'Cradle to Grave', 's

## INTERACTIVE PROGRAM

In [242]:
# Initializes ipywidgets to begin creating interface for the program
# Each input type is defined according to library references, value is extracted from 'I' 
# dictionary which stores input from selected file
widget = ipywidgets.interactive(Completed,
            solver = True, 
            useType = ipywidgets.Dropdown(options=['Long-Distance Aircraft','Short-Distance Aircraft','Articulated Truck','Passenger Car - Diesel', 'Passenger Car - Gasoline'],value=I["useType"],disabled=False), 
            gType = ipywidgets.Dropdown(options=['All','Difference'],value=I["gType"],disabled=False),
            boundry = ipywidgets.Dropdown(options=['Cradle to Grave','Cradle to Gate','Gate to Gate'],value=I["boundry"],disabled=False),
            transpTM = ipywidgets.Dropdown(options=['Ocean Shipping - Diesel','Coastal Shipping - Diesel','Barge - Diesel','Rail - Diesel', 'Articulated HGV (up to 55 metric tons) - Diesel', '40 metric ton truck - Diesel', '32 metric ton truck - Diesel', '14 metric ton truck - Diesel', 'Light goods vehicle - Diesel', 'Family car - Diesel', 'Family car - Gasoline', 'Family car -LPG', 'Family car - Hybrid Gasoline-electric', 'Super sports car and SUV - Gasoline', 'Long Haul Aircraft - Kerosene', 'Short Haul Aircraft - Kerosene', 'Helicopter (Eurocopter AS350) - Kerosene'],value=I["transpTM"],disabled=False),
            transpAM = ipywidgets.Dropdown(options=['Ocean Shipping - Diesel','Coastal Shipping - Diesel','Barge - Diesel','Rail - Diesel', 'Articulated HGV (up to 55 metric tons) - Diesel', '40 metric ton truck - Diesel', '32 metric ton truck - Diesel', '14 metric ton truck - Diesel', 'Light goods vehicle - Diesel', 'Family car - Diesel', 'Family car - Gasoline', 'Family car -LPG', 'Family car - Hybrid Gasoline-electric', 'Super sports car and SUV - Gasoline', 'Long Haul Aircraft - Kerosene', 'Short Haul Aircraft - Kerosene', 'Helicopter (Eurocopter AS350) - Kerosene'],value=I["transpAM"],disabled=False),
            distTM = ipywidgets.FloatText(value=I["distTM"]),
            distAM = ipywidgets.FloatText(value=I["distAM"]),                    
            imassTM =ipywidgets.FloatText(value=I["imassTM"]) , 
            fmassTM =ipywidgets.FloatText(value=I["fmassTM"]) ,
            TM11 = ipywidgets.FloatText(value=I["TM11"]),
            TM12 = ipywidgets.FloatText(value=I["TM12"]),
            TM13 = ipywidgets.FloatSlider(value=I["TM13"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            TM21 = ipywidgets.FloatText(value=I["TM21"]),
            TM22 = ipywidgets.FloatText(value=I["TM22"]),
            TM23 = ipywidgets.FloatSlider(value=I["TM23"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            TM31 = ipywidgets.FloatText(value=I["TM31"]),
            TM32 = ipywidgets.FloatText(value=I["TM32"]),
            TM33 = ipywidgets.FloatSlider(value=I["TM33"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            TM41 = ipywidgets.FloatText(value=I["TM41"]),
            TM42 = ipywidgets.FloatText(value=I["TM42"]),
            TM43 = ipywidgets.FloatSlider(value=I["TM43"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            TM51 = ipywidgets.FloatText(value=I["TM51"]),
            TM52 = ipywidgets.FloatText(value=I["TM52"]),
            TM53 = ipywidgets.FloatSlider(value=I["TM53"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            TMsteps = ipywidgets.IntSlider(value=I["TMsteps"],min=1,max=5,step=1,disabled=False,continuous_update=False,readout=True),
            AM11 = ipywidgets.FloatText(value=I["AM11"]),
            AM12 = ipywidgets.FloatText(value=I["AM12"]),
            AM13 = ipywidgets.FloatSlider(value=I["AM13"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            AM21 = ipywidgets.FloatText(value=I["AM21"]),
            AM22 = ipywidgets.FloatText(value=I["AM22"]),
            AM23 = ipywidgets.FloatSlider(value=I["AM23"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            AM31 = ipywidgets.FloatText(value=I["AM31"]),
            AM32 = ipywidgets.FloatText(value=I["AM32"]),
            AM33 = ipywidgets.FloatSlider(value=I["AM33"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            AM41 = ipywidgets.FloatText(value=I["AM41"]),
            AM42 = ipywidgets.FloatText(value=I["AM42"]),
            AM43 = ipywidgets.FloatSlider(value=I["AM43"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            AM51 = ipywidgets.FloatText(value=I["AM51"]),
            AM52 = ipywidgets.FloatText(value=I["AM52"]),
            AM53 = ipywidgets.FloatSlider(value=I["AM53"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            AMsteps=ipywidgets.IntSlider(value=I["AMsteps"],min=1,max=5,step=1,disabled=False,continuous_update=False,readout=True),
            mass_reduction_perc=ipywidgets.FloatSlider(value=I["mass_reduction_perc"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True), 
            finishing_perc=ipywidgets.FloatSlider(value=I["finishing_perc"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True), 
            support_perc = ipywidgets.FloatSlider(value=I["support_perc"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            eff_energ_conv = ipywidgets.FloatSlider(value=I["eff_energ_conv"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            CO2equivalent = ipywidgets.FloatText(value=I["CO2equivalent"]), 
            recAM=ipywidgets.FloatSlider(value=I["recAM"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True), 
            recTM=ipywidgets.FloatSlider(value=I["recTM"],min=0,max=1,step=0.05,disabled=False,continuous_update=False,readout=True),
            embTMe=ipywidgets.FloatText(value=I["embTMe"]),
            embTMc=ipywidgets.FloatText(value=I["embTMc"]),
            embAMe=ipywidgets.FloatText(value=I["embAMe"]),
            embAMc=ipywidgets.FloatText(value=I["embAMc"]),
            embERecTM=ipywidgets.FloatText(value=I["embERecTM"]),
            embCRecTM=ipywidgets.FloatText(value=I["embCRecTM"]),
            embERecAM=ipywidgets.FloatText(value=I["embERecAM"]),
            embCRecAM=ipywidgets.FloatText(value=I["embCRecAM"]),
            embEDisTM=ipywidgets.FloatText(value=I["embEDisTM"]),
            embCDisTM=ipywidgets.FloatText(value=I["embCDisTM"]),
            embEDisAM=ipywidgets.FloatText(value=I["embEDisAM"]),
            embCDisAM=ipywidgets.FloatText(value=I["embCDisAM"]))


# controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))

# Separates widgets into groups, so that they can be displayed together
mass = HBox(widget.children[0:2], layout = Layout(flex_flow='row wrap'))
materialProperties = HBox(widget.children[2:16], layout = Layout(flex_flow='row wrap'))
manufacturing = HBox(widget.children[16:49], layout = Layout(flex_flow='row wrap'))
percentages = HBox(widget.children[49:52], layout = Layout(flex_flow='row wrap'))
powerGrid = HBox(widget.children[52:54], layout = Layout(flex_flow='row wrap'))
transport1 = HBox(widget.children[54:56], layout = Layout(flex_flow='row wrap'))
transport2 = HBox(widget.children[56:58], layout = Layout(flex_flow='row wrap'))
control = HBox(widget.children[58:62], layout = Layout(flex_flow='row wrap'))

output = widget.children[-1]

# Creates button that will be used to save present input
button = ipywidgets.Button(description="Save input")

# Displays widgets and button and output
display(VBox([mass,materialProperties,manufacturing,percentages,powerGrid,transport1,transport2,control, output, button]))

# Defining button function that receives all widgets and saves their values to a file whose name depends on the present date and time, 
# in order to avoid file names with the same name
def on_button_clicked(b,children=[]):
    
    title = "input"+date.now().strftime("%d-%m-%Y--%H-%M-%S")
    f= open(title,"w+")
    for i in range(len(children)-1):
         f.write(children[i].description + "=" + str(children[i].value)+"\n")
    f.close()
    with output:
        print("Input Saved as " + title+".")

button.on_click(functools.partial(on_button_clicked, children=widget.children))

In [61]:
conta = 0
for i in widget.children:
    
    print(conta,i.description)
    conta += 1


0 imassTM
1 fmassTM
2 embTMe
3 embTMc
4 embAMe
5 embAMc
6 embERecTM
7 embCRecTM
8 embERecAM
9 embCRecAM
10 embEDisTM
11 embCDisTM
12 embEDisAM
13 embCDisAM
14 recAM
15 recTM
16 TM11
17 TM12
18 TM13
19 TM21
20 TM22
21 TM23
22 TM31
23 TM32
24 TM33
25 TM41
26 TM42
27 TM43
28 TM51
29 TM52
30 TM53
31 AM11
32 AM12
33 AM13
34 AM21
35 AM22
36 AM23
37 AM31
38 AM32
39 AM33
40 AM41
41 AM42
42 AM43
43 AM51
44 AM52
45 AM53
46 TMsteps
47 AMsteps
48 mass_reduction_perc
49 finishing_perc
50 support_perc
51 eff_energ_conv
52 CO2equivalent
53 transpTM
54 transpAM
55 distTM
56 distAM
57 useType
58 boundry
59 solver
60 gType


AttributeError: 'Output' object has no attribute 'description'

In [238]:
Calculation(imassTM = 5.13, 
              fmassTM=2.04,
              embTMe=668,
              embTMc=39.5, 
              embAMe=723,
              embAMc=40, 
              embERecTM=500,
              embCRecTM=500,
              embERecAM=500,
              embCRecAM=500,
              embEDisTM=500,
              embCDisTM=500,
              embEDisAM=500,
              embCDisAM=500,
              recAM=0.15, 
              recTM=0.15,
              TM11 = 2.28,
              TM12 = 0.17,
              TM13 = 0.85,
              TM21 = 18.5,
              TM22 = 1.53,
              TM23 = 0.15,
              TM31 = 0,
              TM32 = 0,
              TM33 = 0,
              TM41 = 0,
              TM42 = 0,
              TM43 = 0,
              TM51 = 0,
              TM52 = 0,
              TM53 = 0,
              AM11 = 176.5,
              AM12 = 9.6,
              AM13 = 1,
              AM21 = 18.5,
              AM22 = 1.39,
              AM23 = 0.1,
              AM31 = 0,
              AM32 = 0,
              AM33 = 0,
              AM41 = 0,
              AM42 = 0,
              AM43 = 0,
              AM51 = 0,
              AM52 = 0,
              AM53 = 0, 
              TMsteps=2, 
              AMsteps=2,
              secondary = True,
              mass_reduction_perc=0, 
              finishing_perc=0.1, 
              support_perc = 0.2,
              eff_energ_conv = 0.4,
              CO2equivalent = 20,
              transpTM = 'Ocean Shipping - Diesel',
              transpAM = 'Ocean Shipping - Diesel',
              distTM = 300,
              distAM = 5,
              useType = 'Long-Distance Aircraft', 
              boundry='Cradle to Grave')

({'Material Production': 3426.84,
  'Manufacturing': 60.44422499999999,
  'Transport': 97.92,
  'Use': 0,
  'Disposal': 5477.813999999999,
  'Total': 9063.018225},
 {'Material Production': 202.635,
  'Manufacturing': 483.5537999999999,
  'Transport': 9.18,
  'Use': 0,
  'Disposal': 2737.23975,
  'Total': 3432.60855},
 {'Material Production': 2107.0285714285715,
  'Manufacturing': 1299.407142857143,
  'Transport': 1.632,
  'Use': 0.0,
  'Disposal': 3111.874285714286,
  'Total': 6519.942000000001},
 {'Material Production': 116.57142857142858,
  'Manufacturing': 10395.257142857145,
  'Transport': 0.153,
  'Use': 0.0,
  'Disposal': 1556.2285714285715,
  'Total': 12068.210142857146})